## Introduction

The script seeks to generate data from the https://blog.talosintelligence.com/ website and store both in computer drive as well as in the research postgresql account

In [1]:
#import packages 
import pandas as pd 
import numpy as np
import os 
import itertools

#import time related packages
import time
from datetime import date
from datetime import datetime
from datetime import timedelta


#import webscraping packages 
import bs4 as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
from selenium_stealth import stealth
import undetected_chromedriver as uc

#postgresql database packages 
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine

import csv
from io import StringIO

## Main Page 

In [2]:
#intiate a driver
url = "https://blog.talosintelligence.com/"

driver = webdriver.Chrome(r"C:\Users\chromedriver_win32\chromedriver.exe")#uc.Chrome(use_subprocess=True)
driver.get(url)
driver.maximize_window()
time.sleep(5)


C:\Users\moses\AppData\Local\Temp\ipykernel_21056\1884182325.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\chromedriver_win32\chromedriver.exe")#uc.Chrome(use_subprocess=True)


In [3]:
#create empty dataframe
con_df = pd.DataFrame()

#generate a soup object
soup = bs.BeautifulSoup(driver.page_source,'lxml')

#create a specific tag 
content_divs = soup.find_all('div', {'class': 'date-outer'}) 

#create empty dataframe
con_df = pd.DataFrame()

#loop through the content 
for i in content_divs:
    
    #get the headers 
    hd = i.find_all('h3')
    headers = [x.text.strip() for x in hd][0]
    
    #date of the story 
    dt = i.find_all('h2', {'class': "date-header"})
    date = [x.text.strip() for x in dt][0]
    
    #author 
    au = i.find_all('span', {'class':"post-author"})
    au_tx = [x.text.strip().replace('Posted by', '').replace('\n', '') for x in au][0]
    
    #labels for the story 
    lb = i.find_all('span',{'class':'post-labels'})
    labels = [x.text.strip().replace('Labels:', '').replace('\n', '') for x in lb][0]

    
    #get the blogger url 
    bl = i.find_all('a', {'class': 'g-profile'}, href = True)
    bl_url = [x['href'] for x in bl][0]
    
    #get the truncated story 
    tc_story = {'truncated_story':[]}
    for tc in i.find_all('div', {'class',"post-body entry-content"}):
        tc_tx = [x.text.rstrip().replace('\n', '') for x in tc]
        tc_story['truncated_story'].append(tc_tx)
    
    
    #links 
    links_ls = {'story_link':[]}
    for lk in i.find_all('h3'):
        lk = lk.find_all('a', href = True)
        links = [y['href'] for y in lk][0]
        links_ls['story_link'].append(links)
        
    try:    
        frames = {'published_date':date, 'topics':headers, 'story_link':links_ls.values(), 'posted_by':au_tx, 
                  'labels':labels, 'blogger_url':bl_url, 'truncated_story':tc_story.values()}
        frames_df = pd.DataFrame(frames)
        frames_df['page'] = 1
        frames_df['page url'] = url
        con_df = pd.concat([con_df, frames_df])
    except:
        pass
    
#clean the df 
con_df.reset_index(drop = True, inplace = True)
con_df['story_link'] = con_df['story_link'].apply(lambda x: "".join(x))
con_df['truncated_story'] = con_df['truncated_story'].apply(lambda x: "".join(x[0]))
con_df
    

,published_date,topics,story_link,posted_by,labels,blogger_url,truncated_story,page,page url
0,"Friday, September 2, 2022",Threat Roundup for August 26 to September 2,https://blog.talosintelligence.com/2022/09/thr...,William Largent,Threat Roundup,https://www.blogger.com/profile/12206979422726...,"Today, Talos is publishing a glimpse into the ...",1,https://blog.talosintelligence.com/
1,"Thursday, September 1, 2022","Threat Source newsletter (Sept. 1, 2022) — Con...",https://blog.talosintelligence.com/2022/09/thr...,Jon Munshaw,Threat Source newsletter,https://www.blogger.com/profile/13414456218583...,By Jon Munshaw.Welcome to this week’s edition ...,1,https://blog.talosintelligence.com/
2,"Tuesday, August 30, 2022","ModernLoader delivers multiple stealers, crypt...",https://blog.talosintelligence.com/2022/08/mod...,Vanja Svajcer,"DcRAT,ModernLoader,Redline,SecureX,threats",https://www.blogger.com/profile/18152595187998...,ByVanja SvajcerCisco Talos recently observed t...,1,https://blog.talosintelligence.com/
3,"Friday, August 26, 2022",Threat Roundup for August 19 to August 26,https://blog.talosintelligence.com/2022/08/thr...,William Largent,Threat Roundup,https://www.blogger.com/profile/12206979422726...,"Today, Talos is publishing a glimpse into the ...",1,https://blog.talosintelligence.com/
4,"Thursday, August 25, 2022","Threat Source newsletter (Aug. 25, 2022) — We'...",https://blog.talosintelligence.com/2022/08/thr...,Jon Munshaw,Threat Source newsletter,https://www.blogger.com/profile/13414456218583...,By Jon Munshaw.Welcome to this week’s edition ...,1,https://blog.talosintelligence.com/
5,"Wednesday, August 24, 2022",Ukraine Independence Day: Talos update,https://blog.talosintelligence.com/2022/08/ukr...,Madison Burns,Ukraine,https://www.blogger.com/profile/05703528045231...,"On Independence Day for Ukraine, Aug. 24, 2022...",1,https://blog.talosintelligence.com/
6,"Friday, August 19, 2022",Threat Roundup for August 12 to August 19,https://blog.talosintelligence.com/2022/08/thr...,William Largent,Threat Roundup,https://www.blogger.com/profile/12206979422726...,"Today, Talos is publishing a glimpse into the ...",1,https://blog.talosintelligence.com/
7,"Thursday, August 18, 2022","Threat Source newsletter (Aug. 18, 2022) — Why...",https://blog.talosintelligence.com/2022/08/thr...,Jon Munshaw,Threat Source newsletter,https://www.blogger.com/profile/13414456218583...,By Jon Munshaw.Welcome to this week’s edition ...,1,https://blog.talosintelligence.com/
8,"Tuesday, August 16, 2022",Vulnerability Spotlight: Vulnerabilities in WW...,https://blog.talosintelligence.com/2022/08/vul...,Jon Munshaw,vulnerabilities,https://www.blogger.com/profile/13414456218583...,Claudio Bozzato of Cisco Talos discovered thes...,1,https://blog.talosintelligence.com/


## Click the following pages

In [ ]:
# #create empty dataframe
# url = "https://blog.talosintelligence.com/"
# driver = webdriver.Chrome(r"C:\Users\chromedriver_win32\chromedriver.exe")#uc.Chrome(use_subprocess=True)
# driver.get(url)
# # driver.maximize_window()
# time.sleep(5)

In [ ]:
# #create empty dataframe
# url = "https://blog.talosintelligence.com/"
# driver = webdriver.Chrome(r"C:\Users\chromedriver_win32\chromedriver.exe")#uc.Chrome(use_subprocess=True)
# driver.get(url)
# # driver.maximize_window()
# time.sleep(5)


# #create the df 
# url_dict = {'page':[], 'page_url':[]}

# #intialize the counter 
# start_click = 1
# max_clicks = 1710 #number driven from the total number of post made from 2008 to current time
# interval_click = 1
# count = 0

# #create empty dataframe
# url = "https://blog.talosintelligence.com/"
# driver = webdriver.Chrome(r"C:\Users\chromedriver_win32\chromedriver.exe")#uc.Chrome(use_subprocess=True)
# driver.get(url)
# # driver.maximize_window()
# time.sleep(5)

# #loop through the first 20 pages 
# for i in range(start_click, max_clicks, interval_click):
    
#     #count the number of clicks 
#     count +=1
    
#     #intiate the try clauses
#     try:
#         #click the next pages 
#         older_page = driver.find_element(By.XPATH, '//*[@id="Blog1_blog-pager-older-link"]')
#         older_page.click()
#         time.sleep(5)

#         #get the new page url 
#         new_page_url = driver.current_url
#         print(f"click: {count} | New url > {new_page_url}")
#         url_dict['page'].append(count)
#         url_dict['page_url'].append(new_page_url)
#     except:
#         pass
    
    
# #convert to a dataframe 
# url_df = pd.DataFrame(url_dict)
# url_df

In [5]:
#create empty dataframe
url = "https://blog.talosintelligence.com/"
driver = webdriver.Chrome(r"C:\Users\chromedriver_win32\chromedriver.exe")#uc.Chrome(use_subprocess=True)
driver.get(url)
# driver.maximize_window()
time.sleep(5)

#define the lists
headers_ls =[]
date_ls = []
author_ls = []
blogger_ls = []
labels_ls = []



#create an empty dataframe 
con_df_click = pd.DataFrame()

#intialize the counter 
start_click = 1
max_clicks = 1704 #number driven from the total number of post made from 2008 to current time
interval_click = 1
count = 0

#loop through the first 20 pages 
for i in range(start_click, max_clicks, interval_click):
    
    #count the number of clicks 
    count +=1
    
#     intiate the try clauses
    try:
        #click the next pages 
        older_page = driver.find_element(By.XPATH, '//*[@id="Blog1_blog-pager-older-link"]')
        older_page.click()
        time.sleep(10)

        #get the new page url 
        new_page_url = driver.current_url
        print(f"click: {count} | New url > {new_page_url}")


        driver.get(new_page_url)
    #     driver.maximize_window()
        time.sleep(5)

        #generate a soup object
        soup = bs.BeautifulSoup(driver.page_source,'lxml')

        #create a specific tag 
        content_divs = soup.find_all('div', {'class': 'date-outer'}) 
        #loop through the content 
        for i in content_divs:

            #get the headers 
            try:
                hd = i.find_all('h3')
                headers = [x.text.strip() for x in hd][0]
                headers_ls.append(headers)
            except:
                headers.append('')

            #date of the story 
            try:
                dt = i.find_all('h2', {'class': "date-header"})
                date = [x.text.strip() for x in dt]
                date_ls.append(date)
            except:
                date.append('')

            #author 
            try:
                au = i.find_all('span', {'class':"post-author"})
                au_tx = [x.text.strip().replace('Posted by', '').replace('\n', '') for x in au]
                author_ls.append(au_tx)
            except:
                au_tx.append('')

            #labels for the story 
            try:
                lb = i.find_all('span',{'class':'post-labels'})
                labels = [x.text.strip().replace('Labels:', '').replace('\n', '') for x in lb]
                labels_ls.append(labels)
            except:
                labels.append('')


            #get the blogger url 
            try:
                bl = i.find_all('a', {'class': 'g-profile'}, href = True)
                bl_url = [x['href'] for x in bl]
                blogger_ls.append(bl_url)
            except:
                bl_url.append('')


            #get the truncated story 
            tc_story = {'truncated_story':[]}
            for tc in i.find_all('div', {'class',"post-body entry-content"}):
                tc_tx = [x.text.rstrip().replace('\n', '') for x in tc]
                try:
                    tc_story['truncated_story'].append(tc_tx)
                except:
                    tc_story['truncated_story'].append([])

            #links 
            links_ls = {'story_link':[]} 
            for lk in i.find_all('h3'):
                lk = lk.find_all('a', href = True)
                links = [y['href'] for y in lk]
                try:
                    links_ls['story_link'].append(links)
                except:
                    links_ls['story_link'].append([])


            frames_click = {'topics':headers, 'truncated_story':tc_story.values(), 'story_link':links_ls.values()}
            frames_df_click = pd.DataFrame(frames_click)
            frames_df_click['page'] = count
            frames_df_click['page url'] = new_page_url
            con_df_click = pd.concat([con_df_click, frames_df_click])
            
    except Exception as e:
        print(f'The Exeption was {str(e)}')
        pass
    
    

#clean the df 
con_df_click['story_link'] = con_df_click['story_link'].apply(lambda x: [";".join(lk) for lk in x][0])
# con_df_click['story_link'] = con_df_click['story_link'].apply(lambda x: "".join(x))
# con_df_click['story_link'] = con_df_click['story_link'].apply(lambda x: "".join(x[0][0]))
con_df_click['truncated_story'] = con_df_click['truncated_story'].apply(lambda x: "".join(x[0]))
con_df_click.reset_index(drop = True, inplace = True)
con_df_click.sample(5)

C:\Users\moses\AppData\Local\Temp\ipykernel_21056\3177642125.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\chromedriver_win32\chromedriver.exe")#uc.Chrome(use_subprocess=True)


click: 1 | New url > https://blog.talosintelligence.com/search?updated-max=2022-08-16T11:54:00-04:00&max-results=10
click: 2 | New url > https://blog.talosintelligence.com/search?updated-max=2022-08-04T08:00:00-04:00&max-results=10&start=10&by-date=false
click: 3 | New url > https://blog.talosintelligence.com/search?updated-max=2022-07-27T12:22:00-04:00&max-results=10&start=18&by-date=false
click: 4 | New url > https://blog.talosintelligence.com/search?updated-max=2022-07-13T19:58:00-04:00&max-results=10&start=28&by-date=false
click: 5 | New url > https://blog.talosintelligence.com/search?updated-max=2022-06-21T07:58:00-04:00&max-results=10&start=38&by-date=false
click: 6 | New url > https://blog.talosintelligence.com/search?updated-max=2022-06-01T10:19:00-04:00&max-results=10&start=48&by-date=false
click: 7 | New url > https://blog.talosintelligence.com/search?updated-max=2022-05-13T15:04:00-04:00&max-results=10&start=58&by-date=false
click: 8 | New url > https://blog.talosintelligenc

click: 60 | New url > https://blog.talosintelligence.com/search?updated-max=2020-06-29T13:59:00-04:00&max-results=10&start=578&by-date=false
click: 61 | New url > https://blog.talosintelligence.com/search?updated-max=2020-06-12T18:57:00-04:00&max-results=10&start=588&by-date=false
click: 62 | New url > https://blog.talosintelligence.com/search?updated-max=2020-06-03T10:23:00-04:00&max-results=10&start=598&by-date=false
click: 63 | New url > https://blog.talosintelligence.com/search?updated-max=2020-05-18T13:36:00-04:00&max-results=10&start=608&by-date=false
click: 64 | New url > https://blog.talosintelligence.com/search?updated-max=2020-05-07T14:00:00-04:00&max-results=10&start=618&by-date=false
click: 65 | New url > https://blog.talosintelligence.com/search?updated-max=2020-04-23T11:37:00-04:00&max-results=10&start=628&by-date=false
click: 66 | New url > https://blog.talosintelligence.com/search?updated-max=2020-04-13T11:03:00-04:00&max-results=10&start=638&by-date=false
click: 67 | N

click: 118 | New url > https://blog.talosintelligence.com/search?updated-max=2018-04-02T11:48:00-04:00&max-results=10&start=1148&by-date=false
click: 119 | New url > https://blog.talosintelligence.com/search?updated-max=2018-03-01T10:29:00-05:00&max-results=10&start=1158&by-date=false
click: 120 | New url > https://blog.talosintelligence.com/search?updated-max=2018-02-12T08:16:00-05:00&max-results=10&start=1168&by-date=false
click: 121 | New url > https://blog.talosintelligence.com/search?updated-max=2018-01-18T11:02:00-05:00&max-results=10&start=1178&by-date=false
click: 122 | New url > https://blog.talosintelligence.com/search?updated-max=2018-01-05T14:46:00-05:00&max-results=10&start=1188&by-date=false
click: 123 | New url > https://blog.talosintelligence.com/search?updated-max=2017-12-07T13:06:00-05:00&max-results=10&start=1198&by-date=false
click: 124 | New url > https://blog.talosintelligence.com/search?updated-max=2017-11-13T10:43:00-05:00&max-results=10&start=1208&by-date=false

click: 176 | New url > https://blog.talosintelligence.com/search?updated-max=2011-11-03T15:00:00-04:00&max-results=10&start=1675&by-date=false
click: 177 | New url > https://blog.talosintelligence.com/search?updated-max=2011-07-12T14:45:00-04:00&max-results=10&start=1685&by-date=false
click: 178 | New url > https://blog.talosintelligence.com/search?updated-max=2010-12-23T12:38:00-05:00&max-results=10&start=1695&by-date=false
click: 179 | New url > https://blog.talosintelligence.com/search?updated-max=2010-09-27T17:08:00-04:00&max-results=10&start=1705&by-date=false
click: 180 | New url > https://blog.talosintelligence.com/search?updated-max=2010-08-13T11:26:00-04:00&max-results=10&start=1715&by-date=false
click: 181 | New url > https://blog.talosintelligence.com/search?updated-max=2010-07-15T17:11:00-04:00&max-results=10&start=1725&by-date=false
click: 182 | New url > https://blog.talosintelligence.com/search?updated-max=2010-06-22T12:18:00-04:00&max-results=10&start=1735&by-date=false

...

The Exeption was Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="Blog1_blog-pager-older-link"]"}
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x003F78B3+2193587]
	Ordinal0 [0x00390681+1771137]
	Ordinal0 [0x002A41A8+803240]
	Ordinal0 [0x002D24A0+992416]
	Ordinal0 [0x002D273B+993083]
	Ordinal0 [0x002FF7C2+1177538]
	Ordinal0 [0x002ED7F4+1103860]
	Ordinal0 [0x002FDAE2+1170146]
	Ordinal0 [0x002ED5C6+1103302]
	Ordinal0 [0x002C77E0+948192]
	Ordinal0 [0x002C86E6+952038]
	GetHandleVerifier [0x006A0CB2+2738370]
	GetHandleVerifier [0x006921B8+2678216]
	GetHandleVerifier [0x004817AA+512954]
	GetHandleVerifier [0x00480856+509030]
	Ordinal0 [0x0039743B+1799227]
	Ordinal0 [0x0039BB68+1817448]
	Ordinal0 [0x0039BC55+1817685]
	Ordinal0 [0x003A5230+1856048]
	BaseThreadInitThunk [0x767AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77007B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77007B2E+238]

The Exeption was Message: no such ele

The Exeption was Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="Blog1_blog-pager-older-link"]"}
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x003F78B3+2193587]
	Ordinal0 [0x00390681+1771137]
	Ordinal0 [0x002A41A8+803240]
	Ordinal0 [0x002D24A0+992416]
	Ordinal0 [0x002D273B+993083]
	Ordinal0 [0x002FF7C2+1177538]
	Ordinal0 [0x002ED7F4+1103860]
	Ordinal0 [0x002FDAE2+1170146]
	Ordinal0 [0x002ED5C6+1103302]
	Ordinal0 [0x002C77E0+948192]
	Ordinal0 [0x002C86E6+952038]
	GetHandleVerifier [0x006A0CB2+2738370]
	GetHandleVerifier [0x006921B8+2678216]
	GetHandleVerifier [0x004817AA+512954]
	GetHandleVerifier [0x00480856+509030]
	Ordinal0 [0x0039743B+1799227]
	Ordinal0 [0x0039BB68+1817448]
	Ordinal0 [0x0039BC55+1817685]
	Ordinal0 [0x003A5230+1856048]
	BaseThreadInitThunk [0x767AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77007B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77007B2E+238]

The Exeption was Message: no such ele

The Exeption was Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="Blog1_blog-pager-older-link"]"}
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x003F78B3+2193587]
	Ordinal0 [0x00390681+1771137]
	Ordinal0 [0x002A41A8+803240]
	Ordinal0 [0x002D24A0+992416]
	Ordinal0 [0x002D273B+993083]
	Ordinal0 [0x002FF7C2+1177538]
	Ordinal0 [0x002ED7F4+1103860]
	Ordinal0 [0x002FDAE2+1170146]
	Ordinal0 [0x002ED5C6+1103302]
	Ordinal0 [0x002C77E0+948192]
	Ordinal0 [0x002C86E6+952038]
	GetHandleVerifier [0x006A0CB2+2738370]
	GetHandleVerifier [0x006921B8+2678216]
	GetHandleVerifier [0x004817AA+512954]
	GetHandleVerifier [0x00480856+509030]
	Ordinal0 [0x0039743B+1799227]
	Ordinal0 [0x0039BB68+1817448]
	Ordinal0 [0x0039BC55+1817685]
	Ordinal0 [0x003A5230+1856048]
	BaseThreadInitThunk [0x767AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77007B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77007B2E+238]

The Exeption was Message: no such ele

The Exeption was Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="Blog1_blog-pager-older-link"]"}
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x003F78B3+2193587]
	Ordinal0 [0x00390681+1771137]
	Ordinal0 [0x002A41A8+803240]
	Ordinal0 [0x002D24A0+992416]
	Ordinal0 [0x002D273B+993083]
	Ordinal0 [0x002FF7C2+1177538]
	Ordinal0 [0x002ED7F4+1103860]
	Ordinal0 [0x002FDAE2+1170146]
	Ordinal0 [0x002ED5C6+1103302]
	Ordinal0 [0x002C77E0+948192]
	Ordinal0 [0x002C86E6+952038]
	GetHandleVerifier [0x006A0CB2+2738370]
	GetHandleVerifier [0x006921B8+2678216]
	GetHandleVerifier [0x004817AA+512954]
	GetHandleVerifier [0x00480856+509030]
	Ordinal0 [0x0039743B+1799227]
	Ordinal0 [0x0039BB68+1817448]
	Ordinal0 [0x0039BC55+1817685]
	Ordinal0 [0x003A5230+1856048]
	BaseThreadInitThunk [0x767AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77007B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77007B2E+238]

The Exeption was Message: no such ele

,topics,truncated_story,story_link,page,page url
783,Talos Threat Research Summit Guide and Cisco L...,The first Cisco Talos Threat Research Summit i...,https://blog.talosintelligence.com/2018/06/tal...,114,https://blog.talosintelligence.com/search?upda...
71,Threat Advisory: Spring4Shell,"UPDATE, APRIL 4, 2022:The Kenna Risk Score for...",https://blog.talosintelligence.com/2022/03/thr...,9,https://blog.talosintelligence.com/search?upda...
571,Threat Roundup for August 23 to August 30,"Today, Talos is publishing a glimpse into the ...",https://blog.talosintelligence.com/2019/08/thr...,85,https://blog.talosintelligence.com/search?upda...
1353,The Sudden Reappearance of MS03-039,"Last Friday, I got into the office and pulled ...",https://blog.talosintelligence.com/2010/03/sud...,185,https://blog.talosintelligence.com/search?upda...
293,Interview with a LockBit ransomware operator,"By Azim Khodjibaev, Dymtro Korzhevin and Kenda...",https://blog.talosintelligence.com/2021/02/int...,44,https://blog.talosintelligence.com/search?upda...


In [6]:
print(con_df_click['story_link'][10])

https://blog.talosintelligence.com/2022/08/manjusaka-offensive-framework.html


In [7]:
def find_link():
    for lk in con_df_click['story_link']:
        link = lk[0][0]
        return link

In [ ]:
con_df_click['topics'][1522]

In [9]:
#change the list into a dataframe
main_df = pd.DataFrame({'published_date':date_ls, 'topics':headers_ls, 'posted_by':author_ls, 'labels':labels_ls, 'blogger_url':blogger_ls})

#clean the data
main_df['published_date'] = main_df['published_date'].apply(lambda x: "".join(x))
main_df['topics'] = main_df['topics'].apply(lambda x: "".join(x))
main_df['posted_by'] = main_df['posted_by'].apply(lambda x: "".join(x))
main_df['labels'] = main_df['labels'].apply(lambda x: "".join(x))
main_df['blogger_url'] = main_df['blogger_url'].apply(lambda x: "".join(x))
# main_df['story_link'] = main_df['story_link'].apply(lambda x: "".join(x[0]))
main_df.head(5)

#merge the data 
df = pd.merge(main_df, con_df_click, how = 'left', on = 'topics')

# #drop duplicate s
df.drop_duplicates(subset=['published_date', 'posted_by', 'topics', 'labels'])
df.head(5)

,published_date,topics,posted_by,labels,blogger_url,truncated_story,story_link,page,page url
0,"Tuesday, August 16, 2022",Vulnerability Spotlight: Three vulnerabilities...,Jon Munshaw,vulnerabilities,https://www.blogger.com/profile/13414456218583...,Dave McDaniel of Cisco Talos discovered these ...,https://blog.talosintelligence.com/2022/08/vul...,1,https://blog.talosintelligence.com/search?upda...
1,"Friday, August 12, 2022",Threat Roundup for August 5 to August 12,William Largent,Threat Roundup,https://www.blogger.com/profile/12206979422726...,"Today, Talos is publishing a glimpse into the ...",https://blog.talosintelligence.com/2022/08/thr...,1,https://blog.talosintelligence.com/search?upda...
2,"Thursday, August 11, 2022","Threat Source newsletter (Aug. 11, 2022) — All...",Jon Munshaw,Threat Source newsletter,https://www.blogger.com/profile/13414456218583...,By Jon Munshaw.Welcome to this week’s edition ...,https://blog.talosintelligence.com/2022/08/thr...,1,https://blog.talosintelligence.com/search?upda...
3,"Wednesday, August 10, 2022",Cisco Talos shares insights related to recent ...,Nick Biasini,,https://www.blogger.com/profile/11420644688145...,THIS POST IS ALSO AVAILABLE IN:日本語 (Japanese)U...,https://blog.talosintelligence.com/2022/08/rec...,1,https://blog.talosintelligence.com/search?upda...
4,"Tuesday, August 9, 2022",Microsoft Patch Tuesday for August 2022 — Snor...,Jon Munshaw,"Microsoft Patch Tuesday,vulnerabilities",https://www.blogger.com/profile/13414456218583...,By Jon Munshaw and Vanja Svajcer.Microsoft rel...,https://blog.talosintelligence.com/2022/08/mic...,1,https://blog.talosintelligence.com/search?upda...


In [10]:
df_c = pd.concat([con_df, df])
df_c

,published_date,topics,story_link,posted_by,labels,blogger_url,truncated_story,page,page url
0,"Friday, September 2, 2022",Threat Roundup for August 26 to September 2,https://blog.talosintelligence.com/2022/09/thr...,William Largent,Threat Roundup,https://www.blogger.com/profile/12206979422726...,"Today, Talos is publishing a glimpse into the ...",1,https://blog.talosintelligence.com/
1,"Thursday, September 1, 2022","Threat Source newsletter (Sept. 1, 2022) — Con...",https://blog.talosintelligence.com/2022/09/thr...,Jon Munshaw,Threat Source newsletter,https://www.blogger.com/profile/13414456218583...,By Jon Munshaw.Welcome to this week’s edition ...,1,https://blog.talosintelligence.com/
2,"Tuesday, August 30, 2022","ModernLoader delivers multiple stealers, crypt...",https://blog.talosintelligence.com/2022/08/mod...,Vanja Svajcer,"DcRAT,ModernLoader,Redline,SecureX,threats",https://www.blogger.com/profile/18152595187998...,ByVanja SvajcerCisco Talos recently observed t...,1,https://blog.talosintelligence.com/
3,"Friday, August 26, 2022",Threat Roundup for August 19 to August 26,https://blog.talosintelligence.com/2022/08/thr...,William Largent,Threat Roundup,https://www.blogger.com/profile/12206979422726...,"Today, Talos is publishing a glimpse into the ...",1,https://blog.talosintelligence.com/
4,"Thursday, August 25, 2022","Threat Source newsletter (Aug. 25, 2022) — We'...",https://blog.talosintelligence.com/2022/08/thr...,Jon Munshaw,Threat Source newsletter,https://www.blogger.com/profile/13414456218583...,By Jon Munshaw.Welcome to this week’s edition ...,1,https://blog.talosintelligence.com/
...,...,...,...,...,...,...,...,...,...
1564,"Wednesday, September 3, 2008",AWBO Part Deux,https://blog.talosintelligence.com/2008/09/awb...,Lurene Grenier,"awbo,Defcon,Exploiting",https://www.blogger.com/profile/03409051157167...,Since some people have been chomping at the bi...,204,https://blog.talosintelligence.com/search?upda...
1565,"Friday, August 29, 2008",Checking Multiple Bits in a Flag Field,https://blog.talosintelligence.com/2008/08/che...,Nigel Houghton,"Rules,Snort,Snort Rule Options",https://www.blogger.com/profile/11599266012164...,"Sometimes, it is necessary to check a value in...",204,https://blog.talosintelligence.com/search?upda...
1566,"Friday, August 22, 2008","Defcon, testing and exploiting",https://blog.talosintelligence.com/2008/08/def...,Nigel Houghton,"awbo,Defcon,Exploiting",https://www.blogger.com/profile/11599266012164...,This year at Defcon Immunity trotted out the f...,205,https://blog.talosintelligence.com/search?upda...
1567,"Monday, August 11, 2008",DNS Vulnerability Paper,https://blog.talosintelligence.com/2008/08/dns...,Nigel Houghton,"DNS,Vulnerability,Whitepaper",https://www.blogger.com/profile/11599266012164...,Now that Defcon is over and the Kaminsky DNS V...,205,https://blog.talosintelligence.com/search?upda...


## Name the File 

In [11]:
#generate a filename 
name = f"talosintelligence_log_news_website"

#get todays date
today = datetime.now()

# format the dates
formated_date = today.strftime("%d_%b").lower()

#extension
extension = ".xlsx"

#generate the name
new_filename = name+"_"+formated_date
old_filename = name+"_"+formated_date+"_"+"oldfile"
print(old_filename ,"\n", new_filename)

talosintelligence_log_news_website_06_sep_oldfile 
 talosintelligence_log_news_website_06_sep


## Save the File

In [12]:
#create the connection 
# table_nam
db_host = 'research-dev-db.cluster-c30rc84jwwhk.us-east-2.rds.amazonaws.com'
# db_host = '192.168.8.221'
db_port = 5432
db_password = 'researchpassword'
db = 'postgres'
db_user = 'postgres'


def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db}')

#write the data if the table exists , replace the data
try:
    df_c.to_sql(new_filename, engine, method=psql_insert_copy, if_exists='replace', index= False)
except Exception as e:
    print(f'The error is: {str(e)}')
    pass

## Save in PC Drive

In [13]:
#rename the file if is in directory 
try:
  os.rename(old_filename+'.xlsx', new_filename+'xlsx')
except Exception as e:
  print(f"The error is: {str(e)}")
  pass

The error is: [WinError 2] The system cannot find the file specified: 'talosintelligence_log_news_website_06_sep_oldfile.xlsx' -> 'talosintelligence_log_news_website_06_sepxlsx'


In [14]:
#write the file
try:
    df_c.to_excel(new_filename+'.xlsx', sheet_name = 'blog_data', index = False)
#     coordinates_df.to_excel("african cities coordinates"+'.xlsx', sheet_name = 'tweets data', index = False)
except BaseException as e:
    print('-'*20)
    print(f"The error is {str(e)}")
    pass

In [24]:
print(df_c['blogger_url'][0].reset_index(drop = True)[1])

https://www.blogger.com/profile/13414456218583234191
